In [7]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
%autoreload 2
# from ipython import autoreload
import numpy as np
import causallearn
import random
from clustercausal.algorithms.ClusterPC import ClusterPC
from clustercausal.clusterdag.ClusterDAG import ClusterDAG
from clustercausal.utils.Utils import *

c:\Users\JanMarco\MLProjects\Masterthesis\masterthesis\mthesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# For easier debugging visualization or better understanding the verbose output
# Comment out if not needed and prefer other cluster names
debug_nodes = ['0', '1', '2', '3', '4', '5', '6', '7']
debug_edges = [('0', '1'), ('2', '1'), ('2', '7'), ('1', '6'), ('0','5'), ('3','5'), ('4', '6'), ('3','6'), ('5','6'), ('6','7')]
debug_cluster_mapping = {'X': ['0', '1', '2'], 'Y': ['3', '4'], 'Z': ['5', '6', '7']}
debug_cluster_edges = [('X', 'Z'), ('Y', 'Z')]
debug_no_clust_mapping = {'A' : ['0', '1', '2', '3', '4', '5', '6', '7']}
debug_no_clust_edges = []
nodes = debug_nodes
edges = debug_edges
cluster_mapping = debug_cluster_mapping
cluster_edges = debug_cluster_edges
no_clust_mapping = debug_no_clust_mapping
no_clust_edges = debug_no_clust_edges

In [10]:
# Generate Gaussian ANM
# Example edge_weights, if None is given, they will be generated randomly
# edge_weights = {('X1', 'Z1'): 1, ('Y1', 'Z1'): 2, ('Y1', 'Z2'): 1, ('Z1', 'Z2'): -3, ('Z2', 'Z3'): 2}
edge_weights = None
data, edge_weights = generate_gaussian_anm(nodes, edges, seed = 42, edge_weights = edge_weights)
print(data.shape, edge_weights)
print(data[:4,:])

(10000, 8) {('0', '1'): -3, ('2', '1'): 2, ('2', '7'): 1, ('1', '6'): -1, ('0', '5'): -1, ('3', '5'): 3, ('4', '6'): -3, ('3', '6'): 2, ('5', '6'): -2, ('6', '7'): -3}
[[-1.30217951e+00  3.97213867e+00  1.03677079e+00 -4.21885354e-01
  -1.30193461e+00 -9.67256058e-01 -9.04753175e-01  4.08943541e+00]
 [ 1.27840403e-01  1.08381260e+00  7.31240517e-01  7.26907602e-01
   2.55188152e-01  5.55638406e-01 -7.11890862e-01  2.39881211e+00]
 [-3.16242592e-01 -6.27195581e-01 -1.27923458e-01 -7.37827617e-01
  -7.22138218e-02 -4.66761168e+00  8.00734084e+00 -2.33589462e+01]
 [-1.68011575e-02 -1.63959830e+00 -1.13910746e+00 -2.46998124e+00
  -4.00931718e-01 -8.42940691e+00  1.46208762e+01 -4.55776855e+01]]


In [11]:
#FCI baseline
from causallearn.search.ConstraintBased.FCI import fci

# default parameters
G, edges = fci(data)

# or customized parameters
# G, edges = fci(data, independence_test_method, alpha, depth, max_path_length,
#     verbose, background_knowledge, cache_variables_map)

# visualization
from causallearn.utils.GraphUtils import GraphUtils

pdy = GraphUtils.to_pydot(G)
pdy.write_png('simple_test.png')

Depth=3, working on node 7: 100%|██████████| 8/8 [00:00<00:00, 1000.58it/s]


X7 --> X8


In [12]:
from castle.datasets.simulator import IIDSimulation, DAG
import networkx as nx
from clustercausal.experiments.Simulator import Simulator

In [13]:
import networkx as nx
for i in range(10):
    weighted_random_dag = DAG.erdos_renyi(n_nodes=5, n_edges=200, 
                                      weight_range=(0.5, 2.0), seed=None)
    nx_graph = nx.from_numpy_array(weighted_random_dag, create_using=nx.DiGraph)
    Flag = nx.is_directed_acyclic_graph(nx_graph)
    print(Flag)


True
True
True
True
True
True
True
True
True
True


In [14]:
# method: str, (linear or nonlinear), default='linear'
#         Distribution for standard trainning dataset.
#     sem_type: str
#         gauss, exp, gumbel, uniform, logistic (linear); 
#         mlp, mim, gp, gp-add, quadratic (nonlinear).
#     noise_scale: float
#         Scale parameter of noise distribution in linear SEM.

In [15]:
%load_ext autoreload
%autoreload 2
def test_simulator():
    list_of_dag_nodes = [5, 10, 15, 20]
    list_of_dag_edges = [3, 5, 7, 10]
    list_of_methods = ["erdos_renyi", "scale_free", "bipartite", "hierarchical", "low_rank"]
    list_of_sem_types = ["gauss", "exp", "gumbel", "uniform", "logistic", 
                         "mlp", "mim", "gp", "gp-add", "quadratic"]
    list_of_noise_scales = [0.1, 0.5, 1.0, 2.0]
    for n_nodes in list_of_dag_nodes:
        for n_edges in list_of_dag_edges:
            for method in list_of_methods:
                for sem_type in list_of_sem_types:
                    simulation = Simulator()
                    true_dag, data = simulation.run(true_dag = None, n_nodes = n_nodes, n_edges = n_edges, 
                                                   method=method, weight_range=(-1, 2), 
                                                   distribution_type= sem_type, sample_size = 100,  seed=42, noise_scale = 1.0)
                    nx_graph = nx.from_numpy_array(true_dag.adjacency_matrix, create_using=nx.DiGraph)
                    assert data.shape == (100, n_nodes)
                    assert nx.is_directed_acyclic_graph(nx_graph) == True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
weighted_random_dag = DAG.scale_free(n_nodes=10, n_edges=10, 
                                      weight_range=(0.5, 2.0), seed=1)

In [18]:
from castle.datasets.simulator import IIDSimulation, DAG
weighted_random_dag = DAG.erdos_renyi(n_nodes=10, n_edges=10, 
                                      weight_range=(0.5, 2.0), seed=1)
dataset = IIDSimulation(W=weighted_random_dag, n=2000, method='linear', 
                        sem_type='gauss')

2023-07-12 21:04:56,697 - c:\Users\JanMarco\MLProjects\Masterthesis\masterthesis\mthesis\lib\site-packages\castle\datasets\simulator.py[line:270] - INFO: Finished synthetic dataset


In [19]:
dataset.X

array([[-0.2867778 ,  2.39353195,  1.13069414, ..., -1.10688701,
         2.01687582, -0.00320336],
       [-1.41687082, -1.06307858, -3.00925579, ...,  1.15592087,
         0.42760378,  1.17829491],
       [-0.95379946,  0.53137178, -0.71413117, ...,  0.79099154,
        -0.12547499,  0.73371887],
       ...,
       [ 1.22533869, -1.76505495,  0.92812594, ..., -0.2902744 ,
        -0.34434241,  0.58844553],
       [ 0.23581863,  0.86579349, -0.64452184, ...,  1.11478334,
        -1.02037232,  1.59063352],
       [-0.87610645,  2.16661946, -1.32822531, ...,  0.02340463,
        -0.12653551,  0.59622576]])

In [17]:
test_simulator()

2023-07-12 20:59:09,963 - c:\Users\JanMarco\MLProjects\Masterthesis\masterthesis\mthesis\lib\site-packages\castle\datasets\simulator.py[line:270] - INFO: Finished synthetic dataset


AttributeError: 'IIDSimulation' object has no attribute 'X'

In [ ]:
assert False == True

AssertionError: 